# importando os dados cropped

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
image_path = "drive/My Drive/mascaras/only_faces_images/"


In [ ]:
import cv2 
import os 
import numpy as np

X = []
y = []


for image in os.listdir(image_path):
  try:
    y_ = int(image.split('_')[0])
    x = cv2.imread(image_path + image)
    x = cv2.resize(x, (52,52))

    x = np.asarray(x).astype('float32')
    # print(x.shape)
    y.append(y_)
    X.append(x)
  except:
    print("falhou um")

falhou um
falhou um


# Checando desbalanceamento

In [ ]:
import pandas as pd

pd.Series(y).value_counts()

1    404
0     73
2     25
dtype: int64

# fazendo hot encoder e separando treino e teste (não é necessário para rodar a rede)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y = ohe.fit_transform(np.asarray(y).reshape(-1, 1))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# continuação

In [ ]:
from keras.applications import MobileNetV2

In [ ]:
mbnet = MobileNetV2(
    input_shape=(52,52,3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
for layer in mbnet.layers:
  layer.trainable = False

In [ ]:
import tensorflow as tf

neural_net = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.Conv2D(64, kernel_size = (2,2)),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1,activation='softmax')
])

In [ ]:
neural_net.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 2, 2, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 64)          327744    
                                                                 
 dropout_5 (Dropout)         (None, 1, 1, 64)          0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,585,793
Trainable params: 2,551,681
Non-trainable params: 34,112
_______________________________________

In [ ]:
neural_net.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

In [ ]:
np.asarray(X_train).shape

(336, 52, 52, 3)

In [ ]:
y_train.shape

(502,)

In [ ]:
import numpy as np

neural_net.fit(np.asarray(X), y, epochs = 10)

Epoch 1/10
16/16 [==============================] - 6s 373ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 2/10
16/16 [==============================] - 6s 371ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 3/10
16/16 [==============================] - 6s 376ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 4/10
16/16 [==============================] - 6s 371ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 5/10
16/16 [==============================] - 6s 368ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 6/10
16/16 [==============================] - 6s 374ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 7/10
16/16 [==============================] - 6s 374ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 8/10
16/16 [==============================] - 6s 381ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 9/10
16/16 [==============================] - 7s 452ms/step - loss: 0.0000e+00 - acc: 0.8048
Epoch 10/10
16/16 [==============================] - 6s 395ms/step - loss: 0.0000e+00 - acc: 0.8048


In [ ]:
# x_train = np.asarray(X_train).astype(np.float32)
y = np.asarray(y).astype(np.float32)

In [ ]:
type(X_tr[0][0][0])

numpy.ndarray